In [1]:
import time
import pandas as pd
import numpy as np
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

In [2]:
def init_driver():
    driver = webdriver.Chrome()
    driver.wait = WebDriverWait(driver, 3)
    return driver

In [3]:
def lookup(driver):
    post_dict =  {'link':[], 'title':[], 'stats':[], 'last_post_stats':[]}
    driver.get('http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat')
    driver.find_element_by_xpath('''//*[@id="yui-gen11"]''').click()
    counter1 = 0
    counter2 = 0
    counter3 = 0
    page_number = 1

    while True:
        try:
            link = driver.find_element_by_link_text(str(page_number))
        except NoSuchElementException:
            break

        posts = driver.find_elements_by_xpath('''.//*[@class='rating0 nonsticky']''')
        for post in posts:
            title = post.find_element_by_xpath('''.//a[@class='title']''')
            #print(title.text)
            post_dict['title'].append(title.text)
            #print(title.get_attribute('href'))
            post_dict['link'].append(title.get_attribute('href'))
            #print(post.find_element_by_xpath('''.//*[@class='threadstats td alt']''').text)
            post_dict['stats'].append(post.find_element_by_xpath('''.//*[@class='threadstats td alt']''').text)
            #print(post.find_element_by_xpath('''.//*[@class='threadlastpost td']''').text)
            post_dict['last_post_stats'].append(post.find_element_by_xpath('''.//*[@class='threadlastpost td']''').text)
            counter1 += 1
        print(counter1)

        link.click()
        print(driver.current_url)
        page_number += 1

    return post_dict


In [4]:
def process_df(post_dict):
    post_dict['Last_post_date'] = post_dict.last_post_stats.apply(lambda x:(x.split(',')[0]).split('\n')[1])
    post_dict['Last_post_time'] = post_dict.last_post_stats.apply(lambda x: x.split(',')[1])
    
    post_dict.Last_post_date = post_dict.Last_post_date.apply(lambda x:datetime.datetime.strptime(x,'%m-%d-%Y') if x not in ('Yesterday','Today') else x)
    post_dict.Last_post_time = post_dict.Last_post_time.apply(lambda x:(time.strftime('%H:%M',time.strptime(x,' %I:%M %p'))))
                                                              
    post_dict['Replies'] = post_dict.stats.apply(lambda x: int((x.split('\n')[0]).split(':')[1].replace(',', '')))
    post_dict['Views'] = post_dict.stats.apply(lambda x: int(((x.split('\n')[1]).split(': ')[1]).replace(',','')))
    post_dict.drop(['last_post_stats', 'stats'], axis=1, inplace=True)
    
    return post_dict

In [5]:
def sort_values_by(df, by, ascending):
    sorted_df = df.sort_values(by=by, ascending=ascending)
    return sorted_df

In [6]:
driver = init_driver()
data = lookup(driver)
time.sleep(5)
driver.quit()

18
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat
36
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index2.html
56
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index3.html
76
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index4.html
96
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index5.html
116
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index6.html
134
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index7.html
154
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index8.html
174
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index9.html
194
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index10.html
214
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index11.html
234
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index12.html
254
http://www.f150ecoboost.ne

In [7]:
data = pd.DataFrame.from_dict(data)

In [8]:
data = process_df(data)

In [9]:
data.head()

,link,title,Last_post_date,Last_post_time,Replies,Views
0,http://www.f150ecoboost.net/forum/68-2016-ford...,Lets hear from the happy 2.7 owners that have ...,Yesterday,17:13,19,486
1,http://www.f150ecoboost.net/forum/68-2016-ford...,2106 F150 squeak,Yesterday,07:45,1,54
2,http://www.f150ecoboost.net/forum/68-2016-ford...,Air Conditioning hiccup,2017-08-23 00:00:00,14:28,1,64
3,http://www.f150ecoboost.net/forum/68-2016-ford...,Foam Cover Purpose?,2017-08-17 00:00:00,12:19,3,171
4,http://www.f150ecoboost.net/forum/68-2016-ford...,Speedometer and cruise don't match up,2017-08-10 00:00:00,07:17,9,257


In [10]:
data.dtypes

link              object
title             object
Last_post_date    object
Last_post_time    object
Replies            int64
Views              int64
dtype: object

In [11]:
sorted_data = sort_values_by(data, ['Replies','Views'], [False,False])

In [26]:
sorted_data.head()
type(sorted_data)

pandas.core.frame.DataFrame

In [37]:
a = sorted_data.groupby('Last_post_date').apply(pd.DataFrame.sort_values, 'Replies')

In [38]:
a.head()

,,link,title,Last_post_date,Last_post_time,Replies,Views
Last_post_date,,,,,,,
2015-12-14 00:00:00,266,http://www.f150ecoboost.net/forum/68-2016-ford...,New Member with a 2016 F-150 3.5 Ecoboost.,2015-12-14 00:00:00,17:22,8,2133
2015-12-29 00:00:00,265,http://www.f150ecoboost.net/forum/68-2016-ford...,This section of the forum seems lonely...,2015-12-29 00:00:00,12:12,26,3313
2016-01-15 00:00:00,264,http://www.f150ecoboost.net/forum/68-2016-ford...,New to the Boost,2016-01-15 00:00:00,05:55,6,959
2016-01-23 00:00:00,263,http://www.f150ecoboost.net/forum/68-2016-ford...,"New member, purchasing first eco boost",2016-01-23 00:00:00,11:44,14,2233
2016-01-27 00:00:00,262,http://www.f150ecoboost.net/forum/68-2016-ford...,New member,2016-01-27 00:00:00,11:20,8,601


In [34]:
type(a)

pandas.core.groupby.DataFrameGroupBy

In [39]:
a.to_csv('Top100_Posts.csv')